In [1]:
import pandas as pd
from data_processing import extract_vin_price_pairs, is_valid_vin, transform_data
from catboost import CatBoostRegressor

In [2]:
def predict_car_price(vin_file_path='vin_ford_train.txt', model_path='catboost_model.cbm'):
    # Загружаем обученную модель CatBoost
    model = CatBoostRegressor()
    model.load_model(model_path)
    
    # Читаем пары VIN-кодов и цен из файла
    with open(vin_file_path, 'r') as file:
        vin_price_pairs = [line.strip().split() for line in file]
    
    # Фильтруем невалидные VIN-коды и преобразуем данные в DataFrame
    valid_vin_price_pairs = [(vin, price) for vin, price in vin_price_pairs if is_valid_vin(vin)]
    df = pd.DataFrame(valid_vin_price_pairs, columns=['VIN_code', 'price'])
    
    # Преобразуем данные
    X = df.drop(['price'], axis=1)
    X_transformed = transform_data(X)
    
    # Получаем предсказания
    predictions = model.predict(X_transformed)
    
    # Выводим предсказания
    for vin, prediction in zip(X['VIN_code'], predictions):
        print(f"Для VIN-кода {vin} предсказанная стоимость составляет {prediction}")


<div class="alert alert-warning">
<b>Комментарий ревьюера ⚡:</b> 
Если этот файл ты открыл первым - то основные комментарии в другой тетрадке:)
    
Здесь все хорошо, кроме одного нюанса. 
Ты обращаешься к модели по относительному пути, имея в виду, что модель находится со скриптом в одной папке. Все будет хорошо, пока ты запускаешь скрипт, находясь здесь же, например, так:
 
```bash
python3 function.py
```

Но вот если точка запуска изменится, например, на 

```bash
python3 some_dir/function.py
```

то произойдет страшное. Путь к модели уже не будет найден, хотя ты ничего никуда не перемещал, просто сменил точку запуска интерпретатора.
    
Избежать такого можно, жестко привязав путь к текущей папке где находится скрипт. Например, в твоем случае так
   
    
```python
    model_path=os.path.join(os.path.dirname(__file__), 'catboost_model.cbm')
```
    

</div>

    
